In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.16.0 to work with agogemls


In [2]:
# Get the most recent run of the pipeline
experiment_name = 'titanic_automl_pipeline'
pipeline_experiment = ws.experiments.get(experiment_name)
pipeline_run = list(pipeline_experiment.get_runs())[0]

# Publish the pipeline from the run
published_pipeline = pipeline_run.publish_pipeline(
    name="Titanic_Training_Pipeline", description="Trains titanic model", version="1.0")

published_pipeline

Name,Id,Status,Endpoint
Titanic_Training_Pipeline,b6c2c6cd-23d6-4358-98b9-adeabf586628,Active,REST Endpoint


In [3]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://westeurope.api.azureml.ms/pipelines/v1.0/subscriptions/da21a094-26a3-472f-991b-e2b11979af40/resourceGroups/agoge/providers/Microsoft.MachineLearningServices/workspaces/agogemls/PipelineRuns/PipelineSubmit/b6c2c6cd-23d6-4358-98b9-adeabf586628


In [4]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print("Authentication header ready.")

Authentication header ready.


In [5]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

'f6304f2e-e999-4fbe-ae46-a98246660f87'

In [6]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Submit the Pipeline every Monday at 00:00 UTC
recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")
weekly_schedule = Schedule.create(ws, name="weekly-titanic-training", 
                                  description="Based on time",
                                  pipeline_id=published_pipeline.id, 
                                  experiment_name=experiment_name, 
                                  recurrence=recurrence)
print('Pipeline scheduled.')

Pipeline scheduled.


In [7]:
schedules = Schedule.list(ws)
schedules

[Pipeline(Name: weekly-titanic-training,
 Id: 691e6c10-7277-450d-b77c-630891236a40,
 Status: Active,
 Pipeline Id: b6c2c6cd-23d6-4358-98b9-adeabf586628,
 Recurrence Details: Runs at 0:00 on Monday every Week)]

In [8]:
def stop_by_schedule_id(ws, schedule_id):
    s = next(s for s in Schedule.list(ws) if s.id == schedule_id)
    s.disable()
    return s

schedule_id = '691e6c10-7277-450d-b77c-630891236a40'
stop_by_schedule_id(ws, schedule_id)

Name,Id,Status,Pipeline Id,Recurrence Details
weekly-titanic-training,691e6c10-7277-450d-b77c-630891236a40,Disabled,b6c2c6cd-23d6-4358-98b9-adeabf586628,Runs at 0:00 on Monday every Week


In [9]:
schedules = Schedule.list(ws)
schedules

[]